## Note:
For the most part, I used the following chamecall notebook as a reference https://www.kaggle.com/chamecall/clrp-pretrain

The dataset used was jigsaw-toxic-comment-classification-challenge created by dataista0　https://www.kaggle.com/julian3833/jigsaw-toxic-comment-classification-challenge

Thank you very much!

In [ ]:
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

from transformers import (AutoModel,AutoModelForMaskedLM, 
                          AutoTokenizer, LineByLineTextDataset,
                          DataCollatorForLanguageModeling,
                          Trainer, TrainingArguments)
import re

In [ ]:
train_data = pd.read_csv('../input/jigsaw-toxic-comment-classification-challenge/train.csv')
test_data = pd.read_csv('../input/jigsaw-toxic-comment-classification-challenge/test.csv')

train_data=train_data.loc[:,["id","comment_text"]]
data = pd.concat([train_data,test_data])
data["excerpt"]=data["comment_text"]
data['excerpt'] = data['comment_text'].apply(lambda x: x.replace('\n',''))
def remove_url(sen):
    ret = re.sub(r"(http?|ftp)(:\/\/[-_\.!~*\'()a-zA-Z0-9;\/?:\@&=\+$,%#]+)", "" ,sen)
    return ret

data.excerpt=data.excerpt.map(remove_url)

text  = '\n'.join(data.excerpt.tolist())

with open('text.txt','w') as f:
    f.write(text)

In [ ]:
model_name = "roberta-base"
model = AutoModelForMaskedLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.save_pretrained('./roberta-base');

In [ ]:
train_dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="text.txt", #mention train text file here
    block_size=256)

valid_dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="text.txt", #mention valid text file here
    block_size=256)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15)

training_args = TrainingArguments(
    output_dir="./roberta_base_chk", #select model path for checkpoint
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    evaluation_strategy= 'steps',
    save_total_limit=2,
    eval_steps=500,
    gradient_accumulation_steps=16,
    metric_for_best_model='eval_loss',
    greater_is_better=False,
    load_best_model_at_end =True,
    prediction_loss_only=True,
    report_to = "none")

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset)

In [ ]:
trainer.train()
trainer.save_model('./roberta-base')